In [124]:
import pandas as pd
import nltk
from cleantext import clean
import tqdm
import numpy as np

## Data importation

In [5]:
reviews_df = pd.read_csv("data/reviews.csv", sep=";")

In [9]:
reviews_df

,category_link,company_link,company_name,score,description,review_page_nb,review_link,review_score,review_title,review_text,category
0,https://fr.trustpilot.com/categories/food_beve...,https://fr.trustpilot.com/review/lefourgon.com,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignées ...,329,https://fr.trustpilot.com/reviews/65a5388a60d6...,5,Application conviviale pour passer ses…,Application conviviale pour passer ses command...,food_beverages_tobacco
1,https://fr.trustpilot.com/categories/food_beve...,https://fr.trustpilot.com/review/lefourgon.com,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignées ...,329,https://fr.trustpilot.com/reviews/65a53245a223...,5,Très facile pour la commande,"Très facile pour la commande, très rapide et l...",food_beverages_tobacco
2,https://fr.trustpilot.com/categories/food_beve...,https://fr.trustpilot.com/review/lefourgon.com,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignées ...,329,https://fr.trustpilot.com/reviews/659f0e15dd57...,5,Première expérience réussie !,"Pour nous, c'était une première.Ravis d'avoir ...",food_beverages_tobacco
3,https://fr.trustpilot.com/categories/food_beve...,https://fr.trustpilot.com/review/lefourgon.com,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignées ...,329,https://fr.trustpilot.com/reviews/65a535a6ef47...,5,Livraison parfaite !!,Livraison parfaite !!! Tout est clair et respe...,food_beverages_tobacco
4,https://fr.trustpilot.com/categories/food_beve...,https://fr.trustpilot.com/review/lefourgon.com,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignées ...,329,https://fr.trustpilot.com/reviews/659e5c5b3f1e...,5,Livraison super efficace,"Livraison super efficace, produits de qualités...",food_beverages_tobacco
...,...,...,...,...,...,...,...,...,...,...,...
235498,https://fr.trustpilot.com/categories/vehicles_...,https://fr.trustpilot.com/review/allpneus.com,Allpneus - APN Pneumatiques,"4,6","\nBienvenue chez Allpneus !\nDepuis 2006, ALLP...",76,https://fr.trustpilot.com/reviews/659c57814912...,1,non conforme au descriptif du site,après une première livraison fausse : pas les ...,vehicles_transportation
235499,https://fr.trustpilot.com/categories/vehicles_...,https://fr.trustpilot.com/review/allpneus.com,Allpneus - APN Pneumatiques,"4,6","\nBienvenue chez Allpneus !\nDepuis 2006, ALLP...",76,https://fr.trustpilot.com/reviews/6592e5f3cf8b...,4,presque parfait !! manque la facture !!,"commande et livraison ok, paiement par carte i...",vehicles_transportation
235500,https://fr.trustpilot.com/categories/vehicles_...,https://fr.trustpilot.com/review/allpneus.com,Allpneus - APN Pneumatiques,"4,6","\nBienvenue chez Allpneus !\nDepuis 2006, ALLP...",76,https://fr.trustpilot.com/reviews/65781d1656c0...,4,Parfait sauf le service de livraison,Rien a dire pour ALLPNEUS c'est parfait. Comma...,vehicles_transportation
235501,https://fr.trustpilot.com/categories/vehicles_...,https://fr.trustpilot.com/review/allpneus.com,Allpneus - APN Pneumatiques,"4,6","\nBienvenue chez Allpneus !\nDepuis 2006, ALLP...",76,https://fr.trustpilot.com/reviews/658587017947...,5,j'ai trouvé chez vous la dimension peu…,j'ai trouvé chez vous la dimension peu courant...,vehicles_transportation


## Text Data Cleaning

In [11]:
reviews_df = reviews_df[["company_name", "score", "description", "review_score", "review_title", "review_text", "category"]]

In [12]:
reviews_df

,company_name,score,description,review_score,review_title,review_text,category
0,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignées ...,5,Application conviviale pour passer ses…,Application conviviale pour passer ses command...,food_beverages_tobacco
1,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignées ...,5,Très facile pour la commande,"Très facile pour la commande, très rapide et l...",food_beverages_tobacco
2,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignées ...,5,Première expérience réussie !,"Pour nous, c'était une première.Ravis d'avoir ...",food_beverages_tobacco
3,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignées ...,5,Livraison parfaite !!,Livraison parfaite !!! Tout est clair et respe...,food_beverages_tobacco
4,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignées ...,5,Livraison super efficace,"Livraison super efficace, produits de qualités...",food_beverages_tobacco
...,...,...,...,...,...,...,...
235498,Allpneus - APN Pneumatiques,"4,6","\nBienvenue chez Allpneus !\nDepuis 2006, ALLP...",1,non conforme au descriptif du site,après une première livraison fausse : pas les ...,vehicles_transportation
235499,Allpneus - APN Pneumatiques,"4,6","\nBienvenue chez Allpneus !\nDepuis 2006, ALLP...",4,presque parfait !! manque la facture !!,"commande et livraison ok, paiement par carte i...",vehicles_transportation
235500,Allpneus - APN Pneumatiques,"4,6","\nBienvenue chez Allpneus !\nDepuis 2006, ALLP...",4,Parfait sauf le service de livraison,Rien a dire pour ALLPNEUS c'est parfait. Comma...,vehicles_transportation
235501,Allpneus - APN Pneumatiques,"4,6","\nBienvenue chez Allpneus !\nDepuis 2006, ALLP...",5,j'ai trouvé chez vous la dimension peu…,j'ai trouvé chez vous la dimension peu courant...,vehicles_transportation


cleantext library to remove emojis and line breaks

In [30]:
reviews_df["c"].values[0]

'Le Fourgon vous livre vos boissons consignées à domicile :\n🍶 la commande se passe sur lefourgon.com : bières, jus, sodas, eaux, lait, vins, soupes, spririteux, & co.\n🚚 nous vous livrons gratuitement chez vous sur le créneau choisi\n🌱 et au passage suivant on récupère vos bouteilles vides qu’on renvoie lavées au producteur pour ré-emploi #zerodechet'

In [32]:
clean(reviews_df["description"].values[0], no_emoji=True, no_line_breaks=True, lower=False).replace("#", "")

"Le Fourgon vous livre vos boissons consignees a domicile : la commande se passe sur lefourgon.com : bieres, jus, sodas, eaux, lait, vins, soupes, spririteux, & co. nous vous livrons gratuitement chez vous sur le creneau choisi et au passage suivant on recupere vos bouteilles vides qu'on renvoie lavees au producteur pour re-emploi zerodechet"

In [33]:
reviews_df["description"] = reviews_df["description"].apply(lambda x: clean(x, no_emoji=True, no_line_breaks=True, lower=False).replace("#", ""))

C:\Users\marya\AppData\Local\Temp\ipykernel_31708\2730620396.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df["description"] = reviews_df["description"].apply(lambda x: clean(x, no_emoji=True, no_line_breaks=True, lower=False).replace("#", ""))


In [35]:
reviews_df["description"].values[100]

'"Belleville Brulerie - Paris" est une torrefaction de cafes de qualite en faveur du cafe frais, du cafe filtre, du cafe bodum et des assemblages.'

In [36]:
reviews_df["review_title"] = reviews_df["review_title"].apply(lambda x: clean(x, no_emoji=True, no_line_breaks=True, lower=False).replace("#", ""))

C:\Users\marya\AppData\Local\Temp\ipykernel_31708\3339734242.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df["review_title"] = reviews_df["review_title"].apply(lambda x: clean(x, no_emoji=True, no_line_breaks=True, lower=False).replace("#", ""))


In [37]:
reviews_df["review_text"] = reviews_df["review_text"].apply(lambda x: clean(x, no_emoji=True, no_line_breaks=True, lower=False).replace("#", ""))

C:\Users\marya\AppData\Local\Temp\ipykernel_31708\864621105.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df["review_text"] = reviews_df["review_text"].apply(lambda x: clean(x, no_emoji=True, no_line_breaks=True, lower=False).replace("#", ""))


In [38]:
reviews_df

,company_name,score,description,review_score,review_title,review_text,category
0,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignees ...,5,Application conviviale pour passer ses,Application conviviale pour passer ses command...,food_beverages_tobacco
1,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignees ...,5,Tres facile pour la commande,"Tres facile pour la commande, tres rapide et l...",food_beverages_tobacco
2,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignees ...,5,Premiere experience reussie !,"Pour nous, c'etait une premiere.Ravis d'avoir ...",food_beverages_tobacco
3,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignees ...,5,Livraison parfaite !!,Livraison parfaite !!! Tout est clair et respe...,food_beverages_tobacco
4,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignees ...,5,Livraison super efficace,"Livraison super efficace, produits de qualites...",food_beverages_tobacco
...,...,...,...,...,...,...,...
235498,Allpneus - APN Pneumatiques,"4,6","Bienvenue chez Allpneus ! Depuis 2006, ALLPNEU...",1,non conforme au descriptif du site,apres une premiere livraison fausse : pas les ...,vehicles_transportation
235499,Allpneus - APN Pneumatiques,"4,6","Bienvenue chez Allpneus ! Depuis 2006, ALLPNEU...",4,presque parfait !! manque la facture !!,"commande et livraison ok, paiement par carte i...",vehicles_transportation
235500,Allpneus - APN Pneumatiques,"4,6","Bienvenue chez Allpneus ! Depuis 2006, ALLPNEU...",4,Parfait sauf le service de livraison,Rien a dire pour ALLPNEUS c'est parfait. Comma...,vehicles_transportation
235501,Allpneus - APN Pneumatiques,"4,6","Bienvenue chez Allpneus ! Depuis 2006, ALLPNEU...",5,j'ai trouve chez vous la dimension peu,j'ai trouve chez vous la dimension peu courant...,vehicles_transportation


In [39]:
# reviews_df.to_csv("data/reviews_cleaned_1.csv", sep=";", index=False)

## Translate text

does not work...

In [53]:
from translate import Translator

In [99]:
translator = Translator(from_lang="fr", to_lang="en")
translation = translator.translate("Non conforme au descriptif du site")
translation

'Not in accordance with the site description'

In [71]:
from translate import Translator
translator= Translator(to_lang="fr")
translation = translator.translate("This is a pen.")
translation

"C'est un stylo"

works

In [77]:
from deep_translator import GoogleTranslator

# Use any translator you like, in this example GoogleTranslator
translated = GoogleTranslator(source='auto', target='de').translate("keep it up, you are awesome")  # output -> Weiter so, du bist großartig

In [78]:
translated

'Mach weiter so, du bist großartig'

In [81]:
translated = GoogleTranslator(source='fr', target='en').translate("non conforme au descriptif du site")

In [82]:
translated

'not consistent with the site description'

In [89]:
company_df = reviews_df[["company_name", "score", "description"]].drop_duplicates(subset=["company_name"], ignore_index=True)

In [93]:
company_df["description_len"] = company_df["description"].str.len()

In [109]:
company_df[company_df["description_len"] >= 5000] 

,company_name,score,description,description_len
708,educastream.com,"3,5","Haec igitur Epicuri non probo, inquam. De cete...",8482
1785,ScanMarine TM (Cofrance SARL),"3,8",". SCANMARINE , . . The leading yacht broker in...",7586
3230,EDF ENR,"3,3","Avis sur EDF ENR Filiale du groupe EDF, nous c...",9809
6811,Sofamobili,"1,2","Le 01/08/2017 Le 01/08/2017 Cher Client, Je fa...",5167
7104,ethnik-shop.com,"4,2","Ethnik-Shop, notre Boutique specialisee dans l...",5031
8441,HCOMMEHOME,"2,1","Le 01/08/2017 Cher Client, Je fais suite a vot...",5153


In [97]:
# company_df.to_csv("data/company_desc.csv", sep=";", index=False)

In [91]:
company_df["description_en"] = company_df["description"].apply(lambda x: GoogleTranslator(source='fr', target='en').translate(x))

NotValidLength: Haec igitur Epicuri non probo, inquam. De cetero vellem equidem aut ipse doctrinis fuisset instructior est enim, quod tibi ita videri necesse est, non satis politus iis artibus, quas qui tenent, eruditi appellantur aut ne deterruisset alios a studiis. quamquam te quidem video minime esse deterritum. Homines enim eruditos et sobrios ut infaustos et inutiles vitant, eo quoque accedente quod et nomenclatores adsueti haec et talia venditare, mercede accepta lucris quosdam et prandiis inserunt subditicios ignobiles et obscuros. Homines enim eruditos et sobrios ut infaustos et inutiles vitant, eo quoque accedente quod et nomenclatores adsueti haec et talia venditare, mercede accepta lucris quosdam et prandiis inserunt subditicios ignobiles et obscuros. Constituendi autem sunt qui sint in amicitia fines et quasi termini diligendi. De quibus tres video sententias ferri, quarum nullam probo, unam, ut eodem modo erga amicum adfecti simus, quo erga nosmet ipsos, alteram, ut nostra

In [101]:
translator = Translator(from_lang="fr", to_lang="en")
translation = translator.translate("Non conforme au descriptif du site")
translation

'Not in accordance with the site description'

In [105]:
translated = company_df[company_df["description_len"] > 5000]["description"].apply(lambda x: translator.translate(x))

In [108]:
translated.values

array(['QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS',
       'QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEEDED. MAX ALLOW

In [119]:
list(company_df["description"])

["Le Fourgon vous livre vos boissons consignees a domicile : la commande se passe sur lefourgon.com : bieres, jus, sodas, eaux, lait, vins, soupes, spririteux, & co. nous vous livrons gratuitement chez vous sur le creneau choisi et au passage suivant on recupere vos bouteilles vides qu'on renvoie lavees au producteur pour re-emploi zerodechet",
 'Comptoir des Vignes est une enseigne de caves specialisees en vins, champagnes, spiritueux, bieres de specialites, thes, cafes et epicerie fine. Nos caves se differencient par une mise en valeur originale et moderne des produits, mais egalement sur la base de conseils adaptes aux nouvelles tendances et habitudes de consommation de nos clients. Chaque cave Comptoir des Vignes offre un cadre clair et chaleureux qui permet de decouvrir les vins en toute simplicite et gourmandise avec : Mise en avant des accords Mise a disposition de fiches recettes Animations reguliere en boutique Organisation de soirees degustation Fort de nos 50 caves en France

In [130]:
translator = GoogleTranslator(source='fr', target='en')
# we want to translate all the contents of the "description" column except the ones that are longer than 4999 character
# because the Google Translator API does not support more than 5000 characters
# we will translate the longer ones later
desc_en = []
counter = 0
for desc in list(company_df["description"])[:100]:
    if desc != None:
        if len(desc) > 5000:
            sentences = desc.split(".")
            translated = ""
            for sentence in sentences:
                translated_sentence = translator.translate(sentence)
                if translated_sentence != None:
                    translated += translated_sentence + ". " # we add the "." separator back
            desc_en.append(translated)
        else:
            desc_en.append(translator.translate(desc))
    else:
        desc_en.append(np.nan)
    counter += 1

    if counter % 10 == 0:
        print(f"checkpoint: {counter}")

checkpoint: 10
checkpoint: 20
checkpoint: 30
checkpoint: 40
checkpoint: 50
checkpoint: 60
checkpoint: 70
checkpoint: 80
checkpoint: 90
checkpoint: 100


In [129]:
desc_en

['Le Fourgon delivers your stored drinks to your home: the order is placed on lefourgon.com: beers, juices, sodas, water, milk, wines, soups, spirits, & co. we deliver to your home free of charge in the chosen niche and on the next visit we collect your empty bottles which we return washed to the producer for reuse zerodechet',
 'Comptoir des Vignes is a brand of cellars specializing in wines, champagnes, spirits, specialty beers, teas, coffees and delicatessens. Our cellars are differentiated by an original and modern presentation of the products, but also on the basis of advice adapted to the new trends and consumption habits of our customers. Each Comptoir des Vignes cellar offers a clear and warm setting which allows you to discover the wines in all simplicity and indulgence with: Highlighting the pairings Provision of recipe cards Regular events in store Organization of tasting evenings With our 50 cellars in France , our mission is to satisfy all consumers whatever their needs an

In [131]:
des = pd.read_csv('data/company_desc.csv', sep=";")
des.head()

,company_name,score,description,description_len
0,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignees ...,342
1,Comptoir des Vignes,"4,9",Comptoir des Vignes est une enseigne de caves ...,877
2,Shin Sekai,"4,9",Bienvenue sur notre page Trustpilot ! Shin Sek...,391
3,Nutri Naturel,"4,9","Nutri-Naturel.com, l'epicerie bio en ligne de ...",587
4,Maison Martin - Le Piment Français,"4,9",Maison Martin - Le Piment Francais est la prem...,149


In [157]:
translator = GoogleTranslator(source='fr', target='en')
# we want to translate all the contents of the "description" column except the ones that are longer than 4999 character
# because the Google Translator API does not support more than 5000 characters
# we will translate the longer ones later
desc_en = []
counter = 0
for desc in list(des["description"]):
    if desc != None and isinstance(desc, float) != True:
        if len(desc) > 5000:
            sentences = desc.split(".")
            translated = ""
            for sentence in sentences:
                translated_sentence = translator.translate(sentence)
                if translated_sentence != None:
                    translated += translated_sentence + ". " # we add the "." separator back
            desc_en.append(translated)
        else:
            desc_en.append(translator.translate(desc))
    else:
        desc_en.append(np.nan)
    counter += 1

    if counter % 10 == 0:
        print(f"checkpoint: {counter}")

checkpoint: 10
checkpoint: 20
checkpoint: 30
checkpoint: 40
checkpoint: 50
checkpoint: 60
checkpoint: 70
checkpoint: 80
checkpoint: 90
checkpoint: 100
checkpoint: 110
checkpoint: 120
checkpoint: 130
checkpoint: 140
checkpoint: 150
checkpoint: 160
checkpoint: 170
checkpoint: 180
checkpoint: 190
checkpoint: 200
checkpoint: 210
checkpoint: 220
checkpoint: 230
checkpoint: 240
checkpoint: 250
checkpoint: 260
checkpoint: 270
checkpoint: 280
checkpoint: 290
checkpoint: 300
checkpoint: 310
checkpoint: 320
checkpoint: 330
checkpoint: 340
checkpoint: 350
checkpoint: 360
checkpoint: 370
checkpoint: 380
checkpoint: 390
checkpoint: 400
checkpoint: 410
checkpoint: 420
checkpoint: 430
checkpoint: 440
checkpoint: 450
checkpoint: 460
checkpoint: 470
checkpoint: 480
checkpoint: 490
checkpoint: 500
checkpoint: 510
checkpoint: 520
checkpoint: 530
checkpoint: 540
checkpoint: 550
checkpoint: 560
checkpoint: 570
checkpoint: 580
checkpoint: 590
checkpoint: 600
checkpoint: 610
checkpoint: 620
checkpoint: 630
c

In [159]:
len(desc_en)

12996

In [160]:
import pickle

In [161]:
with open(f"data/desv_en", "wb") as fp:
    pickle.dump(desc_en, fp)

In [162]:
des

,company_name,score,description,description_len
0,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignees ...,342
1,Comptoir des Vignes,"4,9",Comptoir des Vignes est une enseigne de caves ...,877
2,Shin Sekai,"4,9",Bienvenue sur notre page Trustpilot ! Shin Sek...,391
3,Nutri Naturel,"4,9","Nutri-Naturel.com, l'epicerie bio en ligne de ...",587
4,Maison Martin - Le Piment Français,"4,9",Maison Martin - Le Piment Francais est la prem...,149
...,...,...,...,...
12991,Ljbautoparts,"1,2",Vente de Pieces detachees Auto de Carrosserie ...,201
12992,Aéroports de Paris,"1,2","Aeroports de Paris, avec ses trois plates-form...",478
12993,Online SAS,"1,7","Hebergement mutualise avec trafic illimite, no...",83
12994,shopequitation,"1,2",Specialiste on-line de vente de materiel pour ...,269


In [163]:
des["description_en"] = desc_en

In [164]:
des

,company_name,score,description,description_len,description_en
0,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignees ...,342,Le Fourgon delivers your stored drinks to your...
1,Comptoir des Vignes,"4,9",Comptoir des Vignes est une enseigne de caves ...,877,Comptoir des Vignes is a brand of cellars spec...
2,Shin Sekai,"4,9",Bienvenue sur notre page Trustpilot ! Shin Sek...,391,Welcome to our Trustpilot page! Shin Sekai is ...
3,Nutri Naturel,"4,9","Nutri-Naturel.com, l'epicerie bio en ligne de ...",587,"Nutri-Naturel.com, the leading online organic ..."
4,Maison Martin - Le Piment Français,"4,9",Maison Martin - Le Piment Francais est la prem...,149,Maison Martin - Le Piment Francais is the firs...
...,...,...,...,...,...
12991,Ljbautoparts,"1,2",Vente de Pieces detachees Auto de Carrosserie ...,201,"Sale of auto body spare parts online: fender, ..."
12992,Aéroports de Paris,"1,2","Aeroports de Paris, avec ses trois plates-form...",478,"Aeroports de Paris, with its three platforms, ..."
12993,Online SAS,"1,7","Hebergement mutualise avec trafic illimite, no...",83,"Shared hosting with unlimited traffic, domain ..."
12994,shopequitation,"1,2",Specialiste on-line de vente de materiel pour ...,269,Online specialist in the sale of horse riding ...


In [165]:
# des.to_csv("data/company_desc_translated.csv", sep=";", index=False)

## Review Translation

In [153]:
reviews = pd.read_csv('data/reviews_cleaned_1.csv', sep = ";")
reviews.head()

,company_name,score,description,review_score,review_title,review_text,category
0,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignees ...,5,Application conviviale pour passer ses,Application conviviale pour passer ses command...,food_beverages_tobacco
1,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignees ...,5,Tres facile pour la commande,"Tres facile pour la commande, tres rapide et l...",food_beverages_tobacco
2,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignees ...,5,Premiere experience reussie !,"Pour nous, c'etait une premiere.Ravis d'avoir ...",food_beverages_tobacco
3,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignees ...,5,Livraison parfaite !!,Livraison parfaite !!! Tout est clair et respe...,food_beverages_tobacco
4,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignees ...,5,Livraison super efficace,"Livraison super efficace, produits de qualites...",food_beverages_tobacco


In [136]:
reviews_df

,company_name,score,description,review_score,review_title,review_text,category
0,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignees ...,5,Application conviviale pour passer ses,Application conviviale pour passer ses command...,food_beverages_tobacco
1,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignees ...,5,Tres facile pour la commande,"Tres facile pour la commande, tres rapide et l...",food_beverages_tobacco
2,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignees ...,5,Premiere experience reussie !,"Pour nous, c'etait une premiere.Ravis d'avoir ...",food_beverages_tobacco
3,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignees ...,5,Livraison parfaite !!,Livraison parfaite !!! Tout est clair et respe...,food_beverages_tobacco
4,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignees ...,5,Livraison super efficace,"Livraison super efficace, produits de qualites...",food_beverages_tobacco
...,...,...,...,...,...,...,...
235498,Allpneus - APN Pneumatiques,"4,6","Bienvenue chez Allpneus ! Depuis 2006, ALLPNEU...",1,non conforme au descriptif du site,apres une premiere livraison fausse : pas les ...,vehicles_transportation
235499,Allpneus - APN Pneumatiques,"4,6","Bienvenue chez Allpneus ! Depuis 2006, ALLPNEU...",4,presque parfait !! manque la facture !!,"commande et livraison ok, paiement par carte i...",vehicles_transportation
235500,Allpneus - APN Pneumatiques,"4,6","Bienvenue chez Allpneus ! Depuis 2006, ALLPNEU...",4,Parfait sauf le service de livraison,Rien a dire pour ALLPNEUS c'est parfait. Comma...,vehicles_transportation
235501,Allpneus - APN Pneumatiques,"4,6","Bienvenue chez Allpneus ! Depuis 2006, ALLPNEU...",5,j'ai trouve chez vous la dimension peu,j'ai trouve chez vous la dimension peu courant...,vehicles_transportation


In [137]:
translator = GoogleTranslator(source='fr', target='en')
# we want to translate all the contents of the "description" column except the ones that are longer than 4999 character
# because the Google Translator API does not support more than 5000 characters
# we will translate the longer ones later
review_title_en = []
counter = 0
for desc in list(reviews_df["review_title"])[:100]:
    if desc != None and isinstance(desc, float) != True:
        if len(desc) > 5000:
            sentences = desc.split(".")
            translated = ""
            for sentence in sentences:
                translated_sentence = translator.translate(sentence)
                if translated_sentence != None:
                    translated += translated_sentence + ". " # we add the "." separator back
            review_title_en.append(translated)
        else:
            review_title_en.append(translator.translate(desc))
    else:
        review_title_en.append(np.nan)
    counter += 1

    if counter % 10 == 0:
        print(f"checkpoint: {counter}")

checkpoint: 10
checkpoint: 20
checkpoint: 30
checkpoint: 40
checkpoint: 50
checkpoint: 60
checkpoint: 70
checkpoint: 80
checkpoint: 90
checkpoint: 100


In [140]:
review_title_en

['User-friendly application to spend your',
 'Very easy to order',
 'First successful experience!',
 'Perfect delivery!!',
 'Super efficient delivery',
 'Delivery man with a smile despite the cold',
 'No longer having to lug around a bottle',
 'Easy ordering on the app',
 'Top experience',
 'Delivery on time',
 'The delivery people are all adorable',
 'Punctual delivery to my home',
 'Perfect',
 'Well done',
 'I am satisfied',
 'New habit',
 'Varied products',
 'Extra',
 'I love the van!',
 'Travelling expenses',
 'Welcoming store + competent staff',
 'Company found on the way to',
 'vodka',
 'Very dynamic people',
 'Good pick',
 'friendly staff and a great',
 'Christmas shopping',
 'Businesses we like.',
 'Impeccable quality service',
 'Notice',
 'Still so good',
 'Christmas gift',
 'Good advice',
 'Disparaging remarks from the seller',
 'Always an excellent experience',
 "It's a good company that produces wines",
 'positive review',
 'Pleasant store, friendly staff',
 'Very pretty ce

Too many reviews, get only a certain number of reviews for each category

In [139]:
reviews_df["category"].unique()

array(['food_beverages_tobacco', 'animals_pets', 'money_insurance',
       'beauty_wellbeing', 'construction_manufactoring',
       'education_training', 'electronics_technology',
       'events_entertainment', 'hobbies_crafts', 'home_garden',
       'media_publishing', 'restaurants_bars', 'health_medical',
       'utilities', 'home_services', 'business_services',
       'legal_services_government', 'public_local_services',
       'shopping_fashion', 'sports', 'travel_vacation',
       'vehicles_transportation'], dtype=object)

In [141]:
reviews_df["category"].value_counts()

category
home_garden                   26688
shopping_fashion              25348
electronics_technology        19647
money_insurance               17345
events_entertainment          16042
business_services             15698
vehicles_transportation       14442
beauty_wellbeing              14422
food_beverages_tobacco        12149
construction_manufactoring    11581
education_training            10150
hobbies_crafts                10054
travel_vacation                7792
media_publishing               6801
sports                         6619
public_local_services          4013
home_services                  3998
animals_pets                   3678
health_medical                 3645
legal_services_government      3001
restaurants_bars               2070
utilities                       320
Name: count, dtype: int64

In [177]:
reviews_sample = reviews.dropna(subset=["description"])
reviews_sample = reviews_sample.dropna(subset=["review_text", "review_title"])

In [178]:
reviews_sample["category"].value_counts()

category
home_garden                   23822
shopping_fashion              22016
electronics_technology        17244
money_insurance               15213
business_services             14269
events_entertainment          13461
beauty_wellbeing              12725
vehicles_transportation       12223
food_beverages_tobacco        11149
construction_manufactoring    10544
education_training             9565
hobbies_crafts                 9256
travel_vacation                7040
media_publishing               6351
sports                         6120
public_local_services          3852
home_services                  3739
animals_pets                   3292
health_medical                 3289
legal_services_government      2716
restaurants_bars               1746
utilities                       280
Name: count, dtype: int64

In [179]:
reviews_sample = reviews_sample.groupby('category').apply(lambda x: x.sample(n=280, random_state=42)).reset_index(drop=True)
reviews_sample

,company_name,score,description,review_score,review_title,review_text,category
0,Petsonic FR by Guaw,"4,1","Sur notre site web, vous trouverez tout ce que...",3,Prix interessant,Le prix tres interessant pour les croquettes m...,animals_pets
1,Caats,"4,6","Caats, c'est la marque d'alimentation 100% ded...",5,Bonne alternative aux croquettes Royal Canin,Equilibre nutritif pour mon chat et praticite ...,animals_pets
2,Caats,"4,6","Caats, c'est la marque d'alimentation 100% ded...",5,Super produit,"Livraison rapide, prix correct, produits d'exc...",animals_pets
3,Pioui,"4,4",Service de reclamation client gratuit Un seul ...,5,J'ai valide le service d'assistance et,J'ai valide le service d'assistance et depanna...,animals_pets
4,Vétorino,"3,9",Vetorino est un service de boutiques en ligne ...,1,Delais de livraison de plus en plus longs,Au debut les livraisons chez mon veterinaire m...,animals_pets
...,...,...,...,...,...,...,...
6155,Origine Pièces Auto,"4,1",Origine Pieces Auto est une entreprise regiona...,5,Bonjour,"Bonjour, tres satisfait du rapport qualite / p...",vehicles_transportation
6156,Aigle,"1,4",Aigle vous propose sa gamme de vetements de sp...,1,Qualite naufragee,"J'avais herite de bottes Aigle de mon papa, qu...",vehicles_transportation
6157,TURBO-DEPOT,"4,3","Turbo-Depot, ce sont des turbos reconditionnes...",5,plus que satisfais du service de cette,plus que satisfais du service de cette entrepr...,vehicles_transportation
6158,Xenum France,"3,1",Fabricant d'additifs et lubrifiants pour moteu...,1,Xenum France pratique des prix,"Xenum France pratique des prix exorbitant, com...",vehicles_transportation


In [180]:
reviews_sample = reviews_sample.dropna(subset=["review_title", "review_text"])

In [181]:
reviews_sample["category"].value_counts()

category
animals_pets                  280
beauty_wellbeing              280
utilities                     280
travel_vacation               280
sports                        280
shopping_fashion              280
restaurants_bars              280
public_local_services         280
money_insurance               280
media_publishing              280
legal_services_government     280
home_services                 280
home_garden                   280
hobbies_crafts                280
health_medical                280
food_beverages_tobacco        280
events_entertainment          280
electronics_technology        280
education_training            280
construction_manufactoring    280
business_services             280
vehicles_transportation       280
Name: count, dtype: int64

In [182]:
reviews_sample.isna().sum()

company_name    0
score           0
description     0
review_score    0
review_title    0
review_text     0
category        0
dtype: int64

In [183]:
def translate_colunmn(col: list):
    translator = GoogleTranslator(source='fr', target='en')
    # we want to translate all the contents of the "description" column except the ones that are longer than 4999 character
    # because the Google Translator API does not support more than 5000 characters
    # we will translate the longer ones later
    en_translation = []
    counter = 0
    print(len(col))
    for desc in col:
        if desc != None and isinstance(desc, float) != True:
            if len(desc) > 5000:
                sentences = desc.split(".")
                translated = ""
                for sentence in sentences:
                    translated_sentence = translator.translate(sentence)
                    if translated_sentence != None:
                        translated += translated_sentence + ". " # we add the "." separator back
                en_translation.append(translated)
            else:
                en_translation.append(translator.translate(desc))
        else:
            en_translation.append(np.nan)
        counter += 1

        if counter % 10 == 0:
            print(f"checkpoint: {counter}")

    return en_translation

In [184]:
en_title = translate_colunmn(list(reviews_sample["review_title"]))

6160
checkpoint: 10
checkpoint: 20
checkpoint: 30
checkpoint: 40
checkpoint: 50
checkpoint: 60
checkpoint: 70
checkpoint: 80
checkpoint: 90
checkpoint: 100
checkpoint: 110
checkpoint: 120
checkpoint: 130
checkpoint: 140
checkpoint: 150
checkpoint: 160
checkpoint: 170
checkpoint: 180
checkpoint: 190
checkpoint: 200
checkpoint: 210
checkpoint: 220
checkpoint: 230
checkpoint: 240
checkpoint: 250
checkpoint: 260
checkpoint: 270
checkpoint: 280
checkpoint: 290
checkpoint: 300
checkpoint: 310
checkpoint: 320
checkpoint: 330
checkpoint: 340
checkpoint: 350
checkpoint: 360
checkpoint: 370
checkpoint: 380
checkpoint: 390
checkpoint: 400
checkpoint: 410
checkpoint: 420
checkpoint: 430
checkpoint: 440
checkpoint: 450
checkpoint: 460
checkpoint: 470
checkpoint: 480
checkpoint: 490
checkpoint: 500
checkpoint: 510
checkpoint: 520
checkpoint: 530
checkpoint: 540
checkpoint: 550
checkpoint: 560
checkpoint: 570
checkpoint: 580
checkpoint: 590
checkpoint: 600
checkpoint: 610
checkpoint: 620
checkpoint: 

In [185]:
en_title

['Interesting price',
 'Good alternative to Royal Canin kibble',
 'Great product',
 'I have validated the support service and',
 'Increasingly long delivery times',
 'I never received the cat cage',
 'The BLACKHUSKYMALAMUTEAM team spoke to me',
 'huge and beautiful cat tree',
 'Very satisfied at all levels',
 'Beautiful and very good quality',
 'I regularly buy dried worms',
 'Reagent',
 'Very good customer service',
 'Reassuring telephone reception.',
 'true treasure',
 'Your management of my order was',
 'Alright',
 'BOX January 2024',
 'Very bad professionals',
 'Great kibble site',
 'Many thanks for your responsiveness and',
 'Very satisfied with this first purchase',
 'BROKEN ITEM',
 'Everything went well',
 'I had a good experience with',
 'My dog \u200b\u200bis happy to go',
 'I had a very bad experience',
 'pet sitters',
 'Very good site',
 'My chickens are happy',
 'Anti-itch drops',
 'you all answer about the speed of',
 'Correct delivery',
 'I received the hay in bulk in a l

In [186]:
reviews_sample["review_title_en"] = en_title

In [187]:
reviews_sample

,company_name,score,description,review_score,review_title,review_text,category,review_title_en
0,Petsonic FR by Guaw,"4,1","Sur notre site web, vous trouverez tout ce que...",3,Prix interessant,Le prix tres interessant pour les croquettes m...,animals_pets,Interesting price
1,Caats,"4,6","Caats, c'est la marque d'alimentation 100% ded...",5,Bonne alternative aux croquettes Royal Canin,Equilibre nutritif pour mon chat et praticite ...,animals_pets,Good alternative to Royal Canin kibble
2,Caats,"4,6","Caats, c'est la marque d'alimentation 100% ded...",5,Super produit,"Livraison rapide, prix correct, produits d'exc...",animals_pets,Great product
3,Pioui,"4,4",Service de reclamation client gratuit Un seul ...,5,J'ai valide le service d'assistance et,J'ai valide le service d'assistance et depanna...,animals_pets,I have validated the support service and
4,Vétorino,"3,9",Vetorino est un service de boutiques en ligne ...,1,Delais de livraison de plus en plus longs,Au debut les livraisons chez mon veterinaire m...,animals_pets,Increasingly long delivery times
...,...,...,...,...,...,...,...,...
6155,Origine Pièces Auto,"4,1",Origine Pieces Auto est une entreprise regiona...,5,Bonjour,"Bonjour, tres satisfait du rapport qualite / p...",vehicles_transportation,Good morning
6156,Aigle,"1,4",Aigle vous propose sa gamme de vetements de sp...,1,Qualite naufragee,"J'avais herite de bottes Aigle de mon papa, qu...",vehicles_transportation,Shipwreck quality
6157,TURBO-DEPOT,"4,3","Turbo-Depot, ce sont des turbos reconditionnes...",5,plus que satisfais du service de cette,plus que satisfais du service de cette entrepr...,vehicles_transportation,more than satisfied with the service of this
6158,Xenum France,"3,1",Fabricant d'additifs et lubrifiants pour moteu...,1,Xenum France pratique des prix,"Xenum France pratique des prix exorbitant, com...",vehicles_transportation,Xenum France practices prices


In [188]:
en_text = translate_colunmn(list(reviews_sample["review_text"]))

6160
checkpoint: 10
checkpoint: 20
checkpoint: 30
checkpoint: 40
checkpoint: 50
checkpoint: 60
checkpoint: 70
checkpoint: 80
checkpoint: 90
checkpoint: 100
checkpoint: 110
checkpoint: 120
checkpoint: 130
checkpoint: 140
checkpoint: 150
checkpoint: 160
checkpoint: 170
checkpoint: 180
checkpoint: 190
checkpoint: 200
checkpoint: 210
checkpoint: 220
checkpoint: 230
checkpoint: 240
checkpoint: 250
checkpoint: 260
checkpoint: 270
checkpoint: 280
checkpoint: 290
checkpoint: 300
checkpoint: 310
checkpoint: 320
checkpoint: 330
checkpoint: 340
checkpoint: 350
checkpoint: 360
checkpoint: 370
checkpoint: 380
checkpoint: 390
checkpoint: 400
checkpoint: 410
checkpoint: 420
checkpoint: 430
checkpoint: 440
checkpoint: 450
checkpoint: 460
checkpoint: 470
checkpoint: 480
checkpoint: 490
checkpoint: 500
checkpoint: 510
checkpoint: 520
checkpoint: 530
checkpoint: 540
checkpoint: 550
checkpoint: 560
checkpoint: 570
checkpoint: 580
checkpoint: 590
checkpoint: 600
checkpoint: 610
checkpoint: 620
checkpoint: 

In [190]:
len(en_text)

6160

In [191]:
reviews_sample["review_text_en"] = en_text

In [192]:
reviews_sample

,company_name,score,description,review_score,review_title,review_text,category,review_title_en,review_text_en
0,Petsonic FR by Guaw,"4,1","Sur notre site web, vous trouverez tout ce que...",3,Prix interessant,Le prix tres interessant pour les croquettes m...,animals_pets,Interesting price,The price is very interesting for the kibbles ...
1,Caats,"4,6","Caats, c'est la marque d'alimentation 100% ded...",5,Bonne alternative aux croquettes Royal Canin,Equilibre nutritif pour mon chat et praticite ...,animals_pets,Good alternative to Royal Canin kibble,Nutritious balance for my cat and practicality...
2,Caats,"4,6","Caats, c'est la marque d'alimentation 100% ded...",5,Super produit,"Livraison rapide, prix correct, produits d'exc...",animals_pets,Great product,"Fast delivery, fair price, excellent quality p..."
3,Pioui,"4,4",Service de reclamation client gratuit Un seul ...,5,J'ai valide le service d'assistance et,J'ai valide le service d'assistance et depanna...,animals_pets,I have validated the support service and,I have validated the assistance and troublesho...
4,Vétorino,"3,9",Vetorino est un service de boutiques en ligne ...,1,Delais de livraison de plus en plus longs,Au debut les livraisons chez mon veterinaire m...,animals_pets,Increasingly long delivery times,"At first, deliveries to my veterinarian took 4..."
...,...,...,...,...,...,...,...,...,...
6155,Origine Pièces Auto,"4,1",Origine Pieces Auto est une entreprise regiona...,5,Bonjour,"Bonjour, tres satisfait du rapport qualite / p...",vehicles_transportation,Good morning,"Hello, very satisfied with the quality/price r..."
6156,Aigle,"1,4",Aigle vous propose sa gamme de vetements de sp...,1,Qualite naufragee,"J'avais herite de bottes Aigle de mon papa, qu...",vehicles_transportation,Shipwreck quality,"I had inherited Aigle boots from my dad, which..."
6157,TURBO-DEPOT,"4,3","Turbo-Depot, ce sont des turbos reconditionnes...",5,plus que satisfais du service de cette,plus que satisfais du service de cette entrepr...,vehicles_transportation,more than satisfied with the service of this,more than satisfied with the service of this c...
6158,Xenum France,"3,1",Fabricant d'additifs et lubrifiants pour moteu...,1,Xenum France pratique des prix,"Xenum France pratique des prix exorbitant, com...",vehicles_transportation,Xenum France practices prices,"Xenum France charges exorbitant prices, compar..."


In [193]:
reviews_sample.to_csv("data/reviews_sample_translated.csv", sep=";", index=False)